In [2]:
# Import Required Libraries
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pandas as pd

In [3]:
# Image directories
train_dir = "images/training/"
test_dir = "images/test/"

In [4]:
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if not img is None:
                img_list.append((img, label))
    
    return img_list

In [5]:
# Load training data
train_img = load_dataset(train_dir)

In [6]:
import numpy as np

# Ensure train_img is not empty
if len(train_img) > 0:
    # Pick a random index
    pick_random = np.random.randint(0, len(train_img))

    # Check image size
    print(f'Image {pick_random}')
    print(train_img[pick_random][0].shape)



In [7]:
# Function to Visualize
def random_img_viz(img_list):
    rand_num = np.random.randint(0, len(img_list))

    img = img_list[rand_num][0]
    label = img_list[rand_num][1]
    label_str = 'day' if label == 1 else 'night'

    plt.imshow(img)
    print(f'Shape\t: {img.shape}')
    print(f'Label\t: {label}')
    

In [8]:
random_img_viz(train_img)

ValueError: high <= 0